In [1]:
# SETUP - OPTIONAL (USE ONLY IF YOU NEED TO SOURCE THE ALIASES)
source ../test-aliases.bash

In [2]:
# SETUP TEST - STABLE DIFFUSION ALIAS IS WORKING
stable-diffusion | grep "generation"

Image generation via HF Stable Diffusion (SD) API


In [3]:
# SETUP FUNCTION TO FORMAT THE DESCRIPTION OUTPUT
function format_description {
    # Quitar tambien los caracteres especiales como comillas, guiones, etc
    output=$(echo "$1"  | rev | cut -d '.' -f1 | rev | sed "s/, / /g" | sed "s/\"//g" | sed "s/-//g" | sed "s/'//g")
    output=${output:1}
    echo $output
}

In [4]:
# SETUP FUNCTION TO CLEAN GPU MEMORY BEFORE RUN A NEW TEST
function clean_memory {
    echo "Cleaning GPU memory"
    python3 -c "import torch; torch.cuda.empty_cache()"
    nvidia-smi | grep python | awk '{print $3}' | xargs -I {} kill {}
}

In [5]:
# SETUP TEST - STABLE DIFFUSION IMG2TXT ACURACCY
# Format the description after the img2txt command in a example image
description="$(format_description "$(sd-description resources/example.jpg 2> /dev/null)")"
keywords="turtles branch sitting butterfly jungle tree"
not_contained="dog cat angry desert ocean mountain snow city"
echo $description

a group of turtles sitting on a branch flying around the bird elaborated depth of field sparring reallife photograph meditating in lotus position two small horn on the head tilt shift butterflies flying robin eley flying debris long tail


In [6]:
# TEST 1 - STABLE DIFFUSION IMG2TXT ACURACCY (contains some keywords)
echo $description | eval-condition "3 <= len(set(stdin.split()).intersection(set('$keywords'.split())))"

Processing entire input
True


In [7]:
# TEST 2 - STABLE DIFFUSION img2txt ACURACCY (not contain some words)
echo $description | eval-condition "0 == len(set(stdin.split()).intersection(set('$not_contained'.split())))"

Processing entire input
True


In [8]:
# TEST 3 - STABLE DIFFUSION img2img + img2txt ACURACCY
# Original description = $description
clean_memory
stable-diffusion --batch --img2img resources/example.jpg
mv sd-app-image-1.png imgfromimg.png.log
new_description=$(format_description "$(sd-description imgfromimg.png.log 2> /dev/null)")
echo "Original description: $description"
echo "New description: $new_description"
echo $new_description | eval-condition "2 <= len(set(stdin.split()).intersection(set('$new_description'.split())))"


Cleaning GPU memory
/home/geb/.local/lib/python3.11/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/geb/.local/lib/python3.11/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Loading pipeline components...: 100%|█████████████| 7/7 [00:10<00:00,  1.51s/it]
Traceback (most recent call last):
  File "/home/geb/Repos/mezcla/mezcla/examples/hf_stable_diffusion.py", line 1148, in <module>
    main()
  File "/home/geb/Repos/mezcla/mezcla/examples/hf_stable_diffusion.py", line 1122, in main
    else infer_img2img(b64_image_encoding, denoising_factor, prompt, negative_prompt, guidance, skip_img_spec=True))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [9]:
# TEST 4 - STABLE DIFFUSION txt2img + img2txt ACURACCY
stable-diffusion --batch --prompt "$(cat resources/prompt.txt)" --negative "$not_contain" - &>/dev/null
mv sd-app-image-1.png imgfromtxt.png.log
new_description="$(format_description "$(sd-description imgfromtxt.png.log 2> /dev/null)")"
echo "Original description: $description"
echo "New description: $new_description"
echo $description | eval-condition "3 <= len(set(stdin.split()).intersection(set(\"$new_description\".split())))"

Original description: a group of turtles sitting on a branch flying around the bird elaborated depth of field sparring reallife photograph meditating in lotus position two small horn on the head tilt shift butterflies flying robin eley flying debris long tail
New description: two turtles on a branch alamy stock photo rainbow road tilt shift photography three animals speculative evolution by Herman van Swanevelt connections long necks rule of threes potbellied sunny environment panoramic photography status icons vibrant and matching colors petrol photo taken in 2 0 2 0 slowtime
Processing entire input
True
